In [4]:
import pandas as pd
from scipy.interpolate import PchipInterpolator
import numpy as np

In [5]:
df = pd.read_csv('BMED_DATA.csv')

df_ag = pd.DataFrame()

# exp 값을 0부터 시작해서 실험별로 연속적으로 증가하도록 매핑
unique_exps = sorted(df['exp'].unique())
exp_map = {old_exp: new_exp for new_exp, old_exp in enumerate(unique_exps)}

for exp_num in unique_exps:
    exp_data = df[df['exp'] == exp_num]

    # For each experiment, generate new time points at 0.25 intervals up to the end time.
    new_times = np.arange(0, exp_data['time'].max() + 0.25, 0.25)

    # The columns to be interpolated
    columns_to_interpolate = ['VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK','CBK', 'I']

    # exp 값을 0부터 시작해서 연속적으로 증가하도록 저장
    temp_df = pd.DataFrame({
        'exp': exp_map[exp_num],
        'V': exp_data['V'].iloc[0],
        'E': exp_data['E'].iloc[0],
        't': new_times
    })

    # apply the monotonic cubic interpolation for each column
    for col in columns_to_interpolate:
        interpolator = PchipInterpolator(exp_data['time'], exp_data[col])
        temp_df[col] = interpolator(new_times)
    
    # add the results to main dataframe
    if df_ag.empty:
        df_ag = temp_df
    else:
        df_ag = pd.concat([df_ag, temp_df], ignore_index=True)

In [6]:
# save CSV file
df_ag.to_csv('BMED_DATA_AG.csv', index=False)